In [ ]:
%load_ext autoreload
%autoreload 2

import os
import wget
import zipfile
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
# from utils import *
from collections import Counter
from tqdm import tqdm
import time
import geopandas as gpd
import gdown # for downloading files from google drive

# ignore warnings
import warnings
import sys
warnings.filterwarnings("ignore")

In [8]:
def download_dataTMPsets():

    dict = {
        "brightkite": ["https://snap.stanford.edu/data/loc-brightkite_edges.txt.gz", "https://snap.stanford.edu/data/loc-brightkite_totalCheckins.txt.gz"], 
        "gowalla": ["https://snap.stanford.edu/data/loc-gowalla_edges.txt.gz", "https://snap.stanford.edu/data/loc-gowalla_totalCheckins.txt.gz"], 
        "foursquare": "https://drive.google.com/file/d/1PNk3zY8NjLcDiAbzjABzY5FiPAFHq6T8/view?usp=sharing"}

    if not os.path.exists("dataTMP"):
        os.mkdir("dataTMP")
        print("Created dataTMP folder")

    for folder in dict.keys():
        if not os.path.exists(os.path.join("dataTMP", folder)):
            os.mkdir(os.path.join("dataTMP", folder))
            print("Created {} folder".format(folder))

    for folder in dict.keys():
        for url in dict[folder]:
            if folder == "foursquare":
                if not os.path.exists(os.path.join("dataTMP", folder, "foursquare_full.zip")):
                    output = os.path.join("dataTMP", folder, "foursquare_full.zip")
                    gdown.download(url, output, quiet=False, fuzzy=True)
                else :
                    print("{} already downloaded".format(url))
            else:
                if not os.path.exists(os.path.join("dataTMP", folder, url.split("/")[-1])):
                    print("Downloading {}...".format(url))
                    wget.download(url, os.path.join("dataTMP", folder))
                else :
                    print("{} already downloaded".format(url))

    for folder in dict.keys():
        for file in os.listdir(os.path.join("dataTMP", folder)):
            if file.endswith(".gz"):
                print("Unzipping {}...".format(file))
                os.system("gunzip {}".format(os.path.join("dataTMP", folder, file)))
            elif file.endswith(".zip"):
                print("Unzipping {}...".format(file))
                os.system("unzip {}".format(os.path.join("dataTMP", folder, file)))

In [9]:
download_dataTMPsets()

Created dataTMP folder
Created brightkite folder
Created gowalla folder
Created foursquare folder


MissingSchema: Invalid URL 'h': No scheme supplied. Perhaps you meant http://h?

In [ ]:

def download_dataTMPsets():

    urls = [
        ["https://snap.stanford.edu/dataTMP/loc-brightkite_edges.txt.gz", "https://snap.stanford.edu/dataTMP/loc-brightkite_totalCheckins.txt.gz"],
        ["https://snap.stanford.edu/dataTMP/loc-gowalla_edges.txt.gz", "https://snap.stanford.edu/dataTMP/loc-gowalla_totalCheckins.txt.gz"],
        ["https://drive.google.com/file/d/1PNk3zY8NjLcDiAbzjABzY5FiPAFHq6T8/view?usp=sharing"]
    ]

    folders = ["brightkite", "gowalla", "foursquare"]

    if not os.path.exists("dataTMP"):
        os.mkdir("dataTMP")

    for folder in folders:
        if not os.path.exists(os.path.join("dataTMP", folder)):
            os.mkdir(os.path.join("dataTMP", folder))

    # Download every url in their respective folder. For the last one, we have to use gdown, because it's a google drive link. If the file is already downloaded, skip the download

    for i in range(len(urls)):
        for url in urls[i]:
            if not os.path.exists(os.path.join("dataTMP", folders[i], url.split("/")[-1])):
                if i == 2:
                    output = os.path.join("dataTMP", folders[i], "something.zip")
                    gdown.download(url, output, quiet=False, fuzzy=True)
                else:
                    wget.download(url, os.path.join("dataTMP", folders[i]))

download_dataTMPsets()
    # # unzip all the files in the 3 folders. Then remove the .gz or .zip files

    # for folder in folders:
        # for file in os.listdir(os.path.join("dataTMP", folder)):
            # print(folder, file)
            # if file.endswith(".gz"):
                # os.system("gunzip {}".format(os.path.join("dataTMP", folder, file)))
            # elif file.endswith(".zip"):
                # os.system("unzip {}".format(os.path.join("dataTMP", folder, file)))
                # os.remove(os.path.join("dataTMP", folder, file))

    # # take all the .txt files from dataTMP/foursquare/dataTMPset_WWW2019 and move them to dataTMP/foursquare

    # for file in os.listdir(os.path.join("dataTMP", "foursquare", "dataTMPset_WWW2019")):
        # if file.endswith(".txt"):
            # os.rename(os.path.join("dataTMP", "foursquare", "dataTMPset_WWW2019", file), os.path.join("dataTMP", "foursquare", file))

    # # remove the dataTMPset_WWW2019 folder, note that is not empty
    # # os.rmdir(os.path.join("dataTMP", "foursquare", "dataTMPset_WWW2019"))

    # for file in ["dataTMPset_WWW_friendship_old.txt", "dataTMPset_WWW_readme.txt", "raw_Checkins_anonymized.txt", "raw_POIs.txt"]:
        # os.remove(os.path.join("dataTMP", "foursquare", file))

    # # Now we want to clean our dataTMP and rename the files.

    # for file in os.listdir(os.path.join("dataTMP", "brightkite")):
        # if file.endswith("_edges.txt"):
            # os.rename(os.path.join("dataTMP", "brightkite", file), os.path.join("dataTMP", "brightkite", "brightkite_friends_edges.txt"))

    # for file in os.listdir(os.path.join("dataTMP", "gowalla")):
        # if file.endswith("_edges.txt"):
            # os.rename(os.path.join("dataTMP", "gowalla", file), os.path.join("dataTMP", "gowalla", "gowalla_friends_edges.txt"))

    # for file in os.listdir(os.path.join("dataTMP", "foursquare")):
        # if file.endswith("dataTMPset_WWW_friendship_new.txt"):
            # os.rename(os.path.join("dataTMP", "foursquare", file), os.path.join("dataTMP", "foursquare", "foursquare_friends_edges.txt"))

    # # Now we from the _totalCheckins.txt files we want to keep only the first and last column, which are the user ID and the venue ID. We also want to remove the header of the file.

    # for file in os.listdir(os.path.join("dataTMP", "brightkite")):
        # if file.endswith("_totalCheckins.txt"):
            # df = pd.read_csv(os.path.join("dataTMP", "brightkite", file), sep="\t", header=None, names=["user_id", "check-in time", "latitude", "longitude", "venue_id"])
            # df["check-in time"] = pd.to_datetime(df["check-in time"])
            # df = df[df["check-in time"].dt.year == 2010]
            # df = df.drop(["check-in time", "latitude", "longitude"], axis=1)
            # df.to_csv(os.path.join("dataTMP", "brightkite", "brightkite_checkins.txt"), sep="\t", header=False, index=False, errors="ignore", encoding="utf-8")
            # os.remove(os.path.join("dataTMP", "brightkite", file))

    # for file in os.listdir(os.path.join("dataTMP", "gowalla")):
        # if file.endswith("_totalCheckins.txt"):
            # df = pd.read_csv(os.path.join("dataTMP", "gowalla", file), sep="\t", header=None, names=["user_id", "check-in time", "latitude", "longitude", "venue_id"])
            # df["check-in time"] = pd.to_datetime(df["check-in time"])
            # df = df[df["check-in time"].dt.year == 2010]
            # df = df.drop(["check-in time", "latitude", "longitude"], axis=1)
            # df.to_csv(os.path.join("dataTMP", "gowalla", "gowalla_checkins.txt"), sep="\t", header=False, index=False, errors="ignore", encoding="utf-8")
            # os.remove(os.path.join("dataTMP", "gowalla", file))

    # for file in os.listdir(os.path.join("dataTMP", "foursquare")):
        # if file.endswith("dataTMPset_WWW_Checkins_anonymized.txt"):
            # df = pd.read_csv(os.path.join("dataTMP", "foursquare", file), sep="\t", header=None)
            # df = df[[0, 1]]
            # df.to_csv(os.path.join("dataTMP", "foursquare", "foursquare_checkins.txt"), sep="\t", header=False, index=False, errors="ignore", encoding="utf-8")
            # os.remove(os.path.join("dataTMP", "foursquare", file))
